In [1]:
import requests
import pandas as pd
import time
import math

API_KEYS = [
    "468fd187-9460-42b3-adcb-934315fb3c60", 
    "f68ae405-a48a-432d-a4f8-1847ee3afd3c",
    "69969001-6dea-4fa0-8490-533cff7aabcf",
    "66715fc5-9def-486c-8cb7-c6c943833fd1",
    "4614076f-20c1-4632-941c-8a47dec5d386",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "ba1e9466-0f1e-482a-a711-1c2ef8f455fd",
    "06b3535f-186f-4cb7-be6d-14ae08e4b48b"
]

SAVE_INTERVAL = 150  # save every 150 processed rows


def check_number_status(phone_number, api_key, timeout=60):
    url = "https://api.starsender.online/api/check-number"
    headers = {
        "Content-Type": "application/json",
        "Authorization": api_key
    }
    payload = {"number": str(phone_number)}

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()
        r = response.json()
        return r.get("data", {}).get("message", "No message")
    except requests.exceptions.Timeout:
        return "Timeout"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


def process_dataframe(df):
    total_rows = len(df)
    total_keys = len(API_KEYS)

    # Auto batching for API key rotation
    batch_size = math.ceil(total_rows / total_keys)

    print(f"Total rows: {total_rows}")
    print(f"Total API keys: {total_keys}")
    print(f"Auto batch size = {batch_size}\n")

    save_counter = 0  # count rows for autosave

    for i in range(total_rows):
        # Pick API key by row index
        key_index = min(i // batch_size, total_keys - 1)
        current_api_key = API_KEYS[key_index]

        phone_number = df.loc[i, "Whatsapp"]
        result = check_number_status(phone_number, current_api_key)

        df.loc[i, "validity"] = result

        print(f"Row {i+1}/{total_rows} | API {key_index+1}/{total_keys} | {phone_number} → {result}")

        time.sleep(5)
        save_counter += 1

        # Save file every 150 processed rows
        if save_counter >= SAVE_INTERVAL:
            df.to_excel("autosave_result.xlsx", index=False)
            print("💾 Autosaved last 150 rows → autosave_result.xlsx")
            save_counter = 0

    # Final save after finishing all rows
    df.to_excel("final_result.xlsx", index=False)
    print("\n🎉 Done! Saved final_result.xlsx")


# Example
# df = pd.read_excel("input.xlsx")
# process_dataframe(df)


In [2]:
# load data
database = pd.read_excel("Database.xlsx")

# load data validity (consist whatsapp number and the status registered or not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")

In [3]:
no_crm = database[database['CRM'].isna()]
uncheck = no_crm[(no_crm['validity'].isna()) & (no_crm['Source'] == 'Web Ads')].reset_index(drop=True)

In [4]:
process_dataframe(uncheck)

Total rows: 28
Total API keys: 8
Auto batch size = 4

Row 1/28 | API 1/8 | 6281387002275 → Number registered
Row 2/28 | API 1/8 | 6281649115817 → Number registered
Row 3/28 | API 1/8 | 6282349541154 → Number registered
Row 4/28 | API 1/8 | 6285784221769 → Number registered
Row 5/28 | API 2/8 | 6282172779837 → Number registered
Row 6/28 | API 2/8 | 6282371555111 → Number registered
Row 7/28 | API 2/8 | 6281807828191 → Number not registered
Row 8/28 | API 2/8 | 6282280925237 → Number registered
Row 9/28 | API 3/8 | 6285156204490 → Number registered
Row 10/28 | API 3/8 | 6282138555331 → Number registered
Row 11/28 | API 3/8 | 6281262512069 → Number registered
Row 12/28 | API 3/8 | 628127102759 → Number registered
Row 13/28 | API 4/8 | 6285298417516 → Number registered
Row 14/28 | API 4/8 | 6283815988984 → Number registered
Row 15/28 | API 4/8 | 6283818057994 → Number registered
Row 16/28 | API 4/8 | 6287882388588 → Number registered
Row 17/28 | API 5/8 | 6281381027555 → Number registered


In [5]:
# process_dataframe(uncheck)
val_1 = pd.read_excel("final_result.xlsx")
# val_2 = pd.read_excel("batch_2.xlsx")
# val_3 = pd.read_excel("batch_3.xlsx")

In [6]:
validity = pd.concat([val_1])
validity

,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity
0,44937,6281387002275,Faisal,November,2025,NaN,Web Ads,50501,1,PALESTINA,1,07-08,50501,Wed,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number registered
1,44936,6281649115817,50.000,November,2025,NaN,Web Ads,50839,1,PALESTINA,1,08-09,50839,Wed,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number registered
2,44935,6282349541154,Amrah Widana,November,2025,NaN,Web Ads,50209,1,PALESTINA,1,10-11,50209,Sun,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number registered
3,44934,6285784221769,Siti na’imatus sholihah,November,2025,NaN,Web Ads,50679,1,PALESTINA,1,09-10,50679,Wed,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number registered
4,44933,6282172779837,Fitria Widra,November,2025,NaN,Web Ads,100785,1,PALESTINA,1,07-08,100785,Wed,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
5,44932,6282371555111,Meri,November,2025,NaN,Web Ads,53957,1,PALESTINA,1,10-11,53957,Tue,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number registered
6,44931,6281807828191,Tuti Sugiarti,November,2025,NaN,Web Ads,500580,1,PALESTINA,1,06-07,500580,Tue,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number not registered
7,44938,6282280925237,Hamba Allah,November,2025,NaN,Web Ads,100500,1,PALESTINA,1,20-21,100500,Tue,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered
8,44939,6285156204490,Muhammad Fajri Al-Furqon,November,2025,NaN,Web Ads,50436,1,PALESTINA,1,09-10,50436,Wed,Bukan Gajian,2025,Aktif,BRONZE,< 100.000,Number registered
9,44940,6282138555331,Sutrisno harianto,November,2025,NaN,Web Ads,100187,1,PALESTINA,1,12-13,100187,Sun,Bukan Gajian,2025,Aktif,SILVER,100.000 - 1000.000,Number registered


In [7]:
validity_update = pd.concat([data_with_validity, validity])
validity_update = validity_update.drop_duplicates(subset=['Whatsapp'], keep='last').reset_index(drop=True)

In [8]:
validity_update.to_excel("checkpoint_files/data_with_number_validity_updated.xlsx")